# Installs & Imports

In [1]:
!pip install -q opytimizer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.8/206.8 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 234.7/234.7 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.2/56.2 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.6/204.6 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.8/519.8 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.3/276.3 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.0/99.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.3/92.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 50.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.9/468.9 kB 21.5 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from opytimizer.optimizers.science import SA
from opytimizer.spaces import SearchSpace
from opytimizer import Opytimizer
from opytimizer.core import Agent, Function, Optimizer

# Load the Data

In [3]:
# Load the Energy Efficiency dataset
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00242/ENB2012_data.xlsx"
data = pd.read_excel(url)

In [4]:
# Display data
display(data.head())

# Print info
print("\n")
data.info()

,X1,X2,X3,X4,X5,X6,X7,X8,Y1,Y2
0,0.98,514.5,294.0,110.25,7.0,2,0.0,0,15.55,21.33
1,0.98,514.5,294.0,110.25,7.0,3,0.0,0,15.55,21.33
2,0.98,514.5,294.0,110.25,7.0,4,0.0,0,15.55,21.33
3,0.98,514.5,294.0,110.25,7.0,5,0.0,0,15.55,21.33
4,0.90,563.5,318.5,122.50,7.0,2,0.0,0,20.84,28.28




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   X1      768 non-null    float64
 1   X2      768 non-null    float64
 2   X3      768 non-null    float64
 3   X4      768 non-null    float64
 4   X5      768 non-null    float64
 5   X6      768 non-null    int64  
 6   X7      768 non-null    float64
 7   X8      768 non-null    int64  
 8   Y1      768 non-null    float64
 9   Y2      768 non-null    float64
dtypes: float64(8), int64(2)
memory usage: 60.1 KB


# Split the Data

In [5]:
# Select features (X) and target (y)
X = data.iloc[:, :-2].values  # All features except the last two columns
y = data.iloc[:, -2].values.reshape(-1, 1)  # Target: Heating Load (Y1)

In [6]:
# Split the dataset into training, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [7]:
# Print shapes
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("\nX_val shape:", X_val.shape)
print("y_val shape:", y_val.shape)
print("\nX_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

X_train shape: (537, 8)
y_train shape: (537, 1)

X_val shape: (115, 8)
y_val shape: (115, 1)

X_test shape: (116, 8)
y_test shape: (116, 1)


# Scale the Data

In [8]:
# Standardize the features
scaler = StandardScaler().set_output(transform="pandas")
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

# Optimize Weights & Biases

## Standard Model

In [9]:
# Define the neural network class for regression
class TensorFlowNN:
    def __init__(self, input_size, hidden_size):
        self.model = tf.keras.Sequential([
            tf.keras.layers.Input(shape=(input_size,)),
            tf.keras.layers.Dense(hidden_size, activation='relu'),
            tf.keras.layers.Dense(1, activation='linear')  # Single output for regression
        ])

In [10]:
# Model architecture
input_size = X_train.shape[1]
hidden_size = 10

In [11]:
# Train a baseline model using standard optimization with validation set
baseline_model = TensorFlowNN(input_size, hidden_size)
baseline_model.model.compile(optimizer=tf.keras.optimizers.Adam(), loss='mean_squared_error')
baseline_model.model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_val, y_val), verbose=0)

In [12]:
# Evaluate baseline model on the test set
y_pred_baseline = baseline_model.model.predict(X_test)
mse_baseline = np.mean((y_pred_baseline - y_test) ** 2)
print(f"MSE of baseline model (Adam optimizer) on the test set: {mse_baseline:.4f}")

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
MSE of baseline model (Adam optimizer) on the test set: 16.6949


## Opytimizer Model

In [13]:
# Define the model for simulated annealing
model = TensorFlowNN(input_size, hidden_size)

In [14]:
# Calculate the number of trainable parameters
n_weights = int(sum(np.prod(v.shape) for v in model.model.trainable_variables))
print(f"Number of trainable parameters: {n_weights}")

Number of trainable parameters: 101


In [11]:
# Objective function to minimize (Validation MSE)
def objective_function(variables):
    weights = variables  # Extract the weights and biases from the variables
    new_weights = []  # Initialize a list to store the reshaped weights and biases
    start = 0  # Start index for slicing the variables

    for layer in model.model.layers:
        layer_weights = layer.get_weights()  # Get the current weights and biases of the layer
        weight_shape = layer_weights[0].shape  # Shape of the weights for this layer
        bias_shape = layer_weights[1].shape  # Shape of the biases for this layer
        n_weight_params = np.prod(weight_shape)  # Total number of weight parameters
        n_bias_params = np.prod(bias_shape)  # Total number of bias parameters

        # Reshape the slice of weights and biases from variables to match the layer's shapes
        weight_values = weights[start:start + n_weight_params].reshape(weight_shape)
        bias_values = weights[start + n_weight_params:start + n_weight_params + n_bias_params].reshape(bias_shape)

        # Append the reshaped weights and biases to the list
        new_weights.append(weight_values)
        new_weights.append(bias_values)

        start += n_weight_params + n_bias_params  # Update the start index for the next layer

    # Set the model's weights and biases to the new values
    model.model.set_weights(new_weights)

    # Make predictions on the validation set
    y_pred = model.model(X_val, training=False)

    # Calculate the MSE on the validation set
    loss = tf.keras.losses.MeanSquaredError()(y_val, y_pred).numpy()

    return float(loss)  # Return the validation MSE as the objective function value

In [12]:
# Function to set the best weights and biases in the model
def set_best_weights(weights, model):
    new_weights = []  # Initialize a list to store the reshaped weights and biases
    start = 0  # Start index for slicing the weights

    for layer in model.model.layers:
        layer_weights = layer.get_weights()  # Get the current weights and biases of the layer
        weight_shape = layer_weights[0].shape  # Shape of the weights for this layer
        bias_shape = layer_weights[1].shape  # Shape of the biases for this layer
        n_weight_params = np.prod(weight_shape)  # Total number of weight parameters
        n_bias_params = np.prod(bias_shape)  # Total number of bias parameters

        # Reshape the slice of weights and biases from the input to match the layer's shapes
        weight_values = weights[start:start + n_weight_params].reshape(weight_shape)
        bias_values = weights[start + n_weight_params:start + n_weight_params + n_bias_params].reshape(bias_shape)

        # Append the reshaped weights and biases to the list
        new_weights.append(weight_values)
        new_weights.append(bias_values)

        start += n_weight_params + n_bias_params  # Update the start index for the next layer

    # Set the model's weights and biases to the new values
    model.model.set_weights(new_weights)

In [17]:
# Define the search space
lower_bound = -1.0 * np.ones(n_weights)
upper_bound = 1.0 * np.ones(n_weights)

# Set up the SearchSpace
space = SearchSpace(n_agents=1, n_variables=n_weights, lower_bound=lower_bound, upper_bound=upper_bound)

2024-08-23 07:47:56,193 - opytimizer.spaces.search — INFO — Overriding class: Space -> SearchSpace.
2024-08-23 07:47:56,197 - opytimizer.core.space — DEBUG — Agents: 1 | Size: (101, 1) | Lower Bound: [-1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.] | Upper Bound: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1.] | Mapping: ['x0', 'x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8',

In [18]:
# Initialize the function for optimization
function = Function(pointer=objective_function)

# Set simulated annealing hyperparameters
params = {
    't_initial': 0.1,    # Initial temperature
    't_final': 1e-4,     # Final temperature
    'alpha': 0.99        # Cooling rate
}

2024-08-23 07:47:56,209 - opytimizer.core.function — INFO — Creating class: Function.
2024-08-23 07:47:56,213 - opytimizer.core.function — DEBUG — Function: objective_function | Built: True.
2024-08-23 07:47:56,215 - opytimizer.core.function — INFO — Class created.


In [19]:
# Set up the simulated annealing optimizer
optimizer = SA(params=params)

# Bundle everything into the Opytimizer class
opt = Opytimizer(space, optimizer, function)

# Run the optimization process
history = opt.start(n_iterations=10000)

2024-08-23 07:47:56,223 - opytimizer.optimizers.science.sa — INFO — Overriding class: Optimizer -> SA.
2024-08-23 07:47:56,226 - opytimizer.core.optimizer — DEBUG — Algorithm: SA | Custom Parameters: {'t_initial': 0.1, 't_final': 0.0001, 'alpha': 0.99} | Built: True.
2024-08-23 07:47:56,228 - opytimizer.optimizers.science.sa — INFO — Class overrided.
2024-08-23 07:47:56,229 - opytimizer.opytimizer — INFO — Creating class: Opytimizer.
2024-08-23 07:47:56,231 - opytimizer.opytimizer — DEBUG — Space: <opytimizer.spaces.search.SearchSpace object at 0x7e41c81d3b20> | Optimizer: <opytimizer.optimizers.science.sa.SA object at 0x7e41c81d04c0>| Function: <opytimizer.core.function.Function object at 0x7e41c81d3f70>.
2024-08-23 07:47:56,232 - opytimizer.opytimizer — INFO — Class created.
2024-08-23 07:47:56,235 - opytimizer.opytimizer — INFO — Starting optimization task.


100%|##########| 10000/10000 [03:22<00:00, 49.29it/s, fitness=36.8]

2024-08-23 07:51:19,147 - opytimizer.opytimizer — INFO — Optimization task ended.
2024-08-23 07:51:19,149 - opytimizer.opytimizer — INFO — It took 202.91147351264954 seconds.


In [20]:
# Retrieve the best weights
best_weights = opt.space.best_agent.position

# Set the model with the best weights
set_best_weights(best_weights, model)

In [21]:
# Make predictions on the test set
y_pred_sa = model.model.predict(X_test)

# Calculate and print the MSE
mse_sa = np.mean((y_pred_sa - y_test) ** 2)
print(f"MSE of model optimized by Simulated Annealing on the test set: {mse_sa:.4f}")

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
MSE of model optimized by Simulated Annealing on the test set: 39.6970


In [22]:
# Compare results
print("Comparison of MSE:")
print(f"Baseline Model: {mse_baseline:.4f}")
print(f"Simulated Annealing Model: {mse_sa:.4f}")

Comparison of MSE:
Baseline Model: 16.6949
Simulated Annealing Model: 39.6970


# Optimize Hyperparameters

In [13]:
# Objective function for hyperparameter optimization
def objective_function(variables):
    learning_rate = variables[0].item()  # Convert to scalar float
    hidden_units = int(variables[1].item())  # Convert to scalar integer

    # Define the model with the given hyperparameters
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(X_train.shape[1],)),
        tf.keras.layers.Dense(hidden_units, activation='relu'),
        tf.keras.layers.Dense(1, activation='linear')
    ])

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  loss='mean_squared_error')

    # Train the model
    model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val), verbose=0)

    # Evaluate on the validation set
    val_loss = model.evaluate(X_val, y_val, verbose=0)

    return val_loss

In [24]:
# Define the hyperparameters to optimize
n_agents = 5
n_variables = 2  # Number of hyperparameters (learning rate and hidden units)
lower_bound = [1e-4, 5]  # Lower bounds for learning rate and hidden units
upper_bound = [1e-2, 50]  # Upper bounds for learning rate and hidden units

# Define the SearchSpace for hyperparameter optimization
space = SearchSpace(n_agents=n_agents, n_variables=n_variables, lower_bound=lower_bound, upper_bound=upper_bound)

2024-08-23 07:51:19,332 - opytimizer.spaces.search — INFO — Overriding class: Space -> SearchSpace.
2024-08-23 07:51:19,335 - opytimizer.core.space — DEBUG — Agents: 5 | Size: (2, 1) | Lower Bound: [1.e-04 5.e+00] | Upper Bound: [1.e-02 5.e+01] | Mapping: ['x0', 'x1'] | Built: True.
2024-08-23 07:51:19,337 - opytimizer.spaces.search — INFO — Class overrided.


In [25]:
# Initialize the function for optimization
function = Function(pointer=objective_function)

# Set simulated annealing hyperparameters
params = {
    't_initial': 0.1,    # Initial temperature
    't_final': 1e-4,     # Final temperature
    'alpha': 0.99        # Cooling rate
}

2024-08-23 07:51:19,345 - opytimizer.core.function — INFO — Creating class: Function.
2024-08-23 07:51:19,348 - opytimizer.core.function — DEBUG — Function: objective_function | Built: True.
2024-08-23 07:51:19,349 - opytimizer.core.function — INFO — Class created.


In [26]:
# Set up the simulated annealing optimizer
optimizer = SA(params=params)

# Bundle everything into the Opytimizer class
opt = Opytimizer(space, optimizer, function)

# Run the optimization process
history = opt.start(n_iterations=100)

2024-08-23 07:51:19,358 - opytimizer.optimizers.science.sa — INFO — Overriding class: Optimizer -> SA.
2024-08-23 07:51:19,360 - opytimizer.core.optimizer — DEBUG — Algorithm: SA | Custom Parameters: {'t_initial': 0.1, 't_final': 0.0001, 'alpha': 0.99} | Built: True.
2024-08-23 07:51:19,362 - opytimizer.optimizers.science.sa — INFO — Class overrided.
2024-08-23 07:51:19,364 - opytimizer.opytimizer — INFO — Creating class: Opytimizer.
2024-08-23 07:51:19,365 - opytimizer.opytimizer — DEBUG — Space: <opytimizer.spaces.search.SearchSpace object at 0x7e41c83ee680> | Optimizer: <opytimizer.optimizers.science.sa.SA object at 0x7e41c83ee830>| Function: <opytimizer.core.function.Function object at 0x7e41c83ed060>.
2024-08-23 07:51:19,366 - opytimizer.opytimizer — INFO — Class created.
2024-08-23 07:51:19,369 - opytimizer.opytimizer — INFO — Starting optimization task.


100%|##########| 100/100 [1:14:30<00:00, 44.71s/it, fitness=4.06]

2024-08-23 09:06:11,433 - opytimizer.opytimizer — INFO — Optimization task ended.
2024-08-23 09:06:11,434 - opytimizer.opytimizer — INFO — It took 4492.063373088837 seconds.


In [27]:
# Retrieve the best hyperparameters
best_hyperparams = opt.space.best_agent.position

# Extract the scalar values from the numpy arrays
learning_rate = best_hyperparams[0].item()  # Convert to float
hidden_units = int(best_hyperparams[1].item())  # Convert to integer

# Print the best hyperparameters found
print(f"Best hyperparameters found: Learning Rate = {learning_rate:.6f}, Hidden Units = {hidden_units}")

Best hyperparameters found: Learning Rate = 0.010000, Hidden Units = 40


In [28]:
# Train the final model using the best hyperparameters
final_model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(hidden_units, activation='relu'),
    tf.keras.layers.Dense(1, activation='linear')
])

final_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                    loss='mean_squared_error')

final_model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val), verbose=0)

In [29]:
# Evaluate the optimized model on the test set
y_pred_sa = final_model.predict(X_test)
mse_sa = np.mean((y_pred_sa - y_test) ** 2)
print(f"MSE of optimized model (Simulated Annealing) on the test set: {mse_sa:.4f}")

1/4 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
MSE of optimized model (Simulated Annealing) on the test set: 3.6777


# Optimize Both

In [30]:
# Define the search space for hyperparameters
lower_bound_hyperparams = np.array([5])    # Lower bounds for hidden units
upper_bound_hyperparams = np.array([50])   # Upper bounds for hidden units

n_variables_hyperparams = len(lower_bound_hyperparams)

In [31]:
# Set up the SearchSpace for hyperparameter optimization
space_hyperparams = SearchSpace(
    n_agents=5,
    n_variables=n_variables_hyperparams,
    lower_bound=lower_bound_hyperparams,
    upper_bound=upper_bound_hyperparams
)

2024-08-23 09:06:15,935 - opytimizer.spaces.search — INFO — Overriding class: Space -> SearchSpace.
2024-08-23 09:06:15,939 - opytimizer.core.space — DEBUG — Agents: 5 | Size: (1, 1) | Lower Bound: [5] | Upper Bound: [50] | Mapping: ['x0'] | Built: True.
2024-08-23 09:06:15,941 - opytimizer.spaces.search — INFO — Class overrided.


In [14]:
# Function to optimize weights and biases for a given set of hyperparameters
def weight_objective_function(weights, model, X_train, y_train, X_val, y_val):
    # Set the weights and biases in the model
    new_weights = []
    start = 0

    for layer in model.model.layers:
        layer_weights = layer.get_weights()
        weight_shape = layer_weights[0].shape
        bias_shape = layer_weights[1].shape
        n_weight_params = np.prod(weight_shape)
        n_bias_params = np.prod(bias_shape)

        # Extract and reshape weights and biases
        weight_values = weights[start:start + n_weight_params].reshape(weight_shape)
        bias_values = weights[start + n_weight_params:start + n_weight_params + n_bias_params].reshape(bias_shape)

        new_weights.append(weight_values)
        new_weights.append(bias_values)

        start += n_weight_params + n_bias_params

    model.model.set_weights(new_weights)

    # Evaluate the model on the validation set
    y_val_pred = model.model.predict(X_val, verbose=0)
    val_loss = tf.keras.losses.MeanSquaredError()(y_val, y_val_pred).numpy()

    return float(val_loss)  # Ensure the return value is a float

In [15]:
# Function to set the best weights
def set_best_weights(weights, model):
    new_weights = []
    start = 0
    for layer in model.model.layers:
        layer_weights = layer.get_weights()
        weight_shape = layer_weights[0].shape
        bias_shape = layer_weights[1].shape
        n_weight_params = np.prod(weight_shape)
        n_bias_params = np.prod(bias_shape)

        # Extract and reshape weights and biases
        weight_values = weights[start:start + n_weight_params].reshape(weight_shape)
        bias_values = weights[start + n_weight_params:start + n_weight_params + n_bias_params].reshape(bias_shape)

        new_weights.append(weight_values)
        new_weights.append(bias_values)

        start += n_weight_params + n_bias_params

    # Set the weights and biases in the model
    model.model.set_weights(new_weights)

In [16]:
# Outer function to optimize hyperparameters and invoke the inner loop
def hyperparam_objective_function(variables):
    # Extract hyperparameters
    hidden_units = int(variables[0].item())

    # Initialize the model with the given hyperparameters
    model = TensorFlowNN(input_size=X_train.shape[1], hidden_size=hidden_units)

    # Define the search space for weights and biases
    n_weights = int(sum(np.prod(v.shape) for v in model.model.trainable_variables))
    lower_bound_weights = -1.0 * np.ones(n_weights)
    upper_bound_weights = 1.0 * np.ones(n_weights)

    # Set up the SearchSpace for weight and bias optimization
    space_weights = SearchSpace(
        n_agents=5,
        n_variables=n_weights,
        lower_bound=lower_bound_weights,
        upper_bound=upper_bound_weights
    )

    # Define the inner objective function for weights optimization
    function_weights = Function(
        pointer=lambda weights: weight_objective_function(weights, model, X_train, y_train, X_val, y_val)
    )

    # Set up the simulated annealing optimizer for weights and biases
    params_inner = {
        't_initial': 0.1,
        't_final': 1e-4,
        'alpha': 0.99
    }
    optimizer_weights = SA(params=params_inner)
    opt_weights = Opytimizer(space_weights, optimizer_weights, function_weights)

    # Run the inner optimization process for weights and biases
    history_weights = opt_weights.start(n_iterations=20)

    # Retrieve the best weights found by the inner optimization
    best_weights = opt_weights.space.best_agent.position

    # Set the best weights and biases on the model
    set_best_weights(best_weights, model)

    # Use the optimized model to compute the validation loss
    y_val_pred = model.model.predict(X_val)
    val_loss = tf.keras.losses.MeanSquaredError()(y_val, y_val_pred).numpy()

    # Return the validation loss as the objective value
    return float(val_loss)

In [35]:
# Set up the simulated annealing optimizer for hyperparameters
params_outer = {
    't_initial': 0.1,
    't_final': 1e-4,
    'alpha': 0.99
}
optimizer_hyperparams = SA(params=params_outer)
function_hyperparams = Function(pointer=hyperparam_objective_function)
opt_hyperparams = Opytimizer(space_hyperparams, optimizer_hyperparams, function_hyperparams)

# Run the outer optimization process for hyperparameters
history_hyperparams = opt_hyperparams.start(n_iterations=10)

2024-08-23 09:06:15,974 - opytimizer.optimizers.science.sa — INFO — Overriding class: Optimizer -> SA.
2024-08-23 09:06:15,975 - opytimizer.core.optimizer — DEBUG — Algorithm: SA | Custom Parameters: {'t_initial': 0.1, 't_final': 0.0001, 'alpha': 0.99} | Built: True.
2024-08-23 09:06:15,977 - opytimizer.optimizers.science.sa — INFO — Class overrided.
2024-08-23 09:06:15,980 - opytimizer.core.function — INFO — Creating class: Function.
2024-08-23 09:06:15,981 - opytimizer.core.function — DEBUG — Function: hyperparam_objective_function | Built: True.
2024-08-23 09:06:15,982 - opytimizer.core.function — INFO — Class created.
2024-08-23 09:06:15,984 - opytimizer.opytimizer — INFO — Creating class: Opytimizer.
2024-08-23 09:06:15,987 - opytimizer.opytimizer — DEBUG — Space: <opytimizer.spaces.search.SearchSpace object at 0x7e40878b6a10> | Optimizer: <opytimizer.optimizers.science.sa.SA object at 0x7e408781e4d0>| Function: <opytimizer.core.function.Function object at 0x7e408781f970>.
2024-08

100%|##########| 20/20 [00:15<00:00,  1.27it/s, fitness=535]

2024-08-23 09:06:32,251 - opytimizer.opytimizer — INFO — Optimization task ended.
2024-08-23 09:06:32,252 - opytimizer.opytimizer — INFO — It took 16.20983600616455 seconds.
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
2024-08-23 09:06:32,357 - opytimizer.spaces.search — INFO — Overriding class: Space -> SearchSpace.
2024-08-23 09:06:32,361 - opytimizer.core.space — DEBUG — Agents: 5 | Size: (61, 1) | Lower Bound: [-1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1.] | Upper Bound: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.] | Mapping: ['x0', 'x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10', 'x11', 'x12', 'x13', 'x14', 'x15', 'x16', 'x17', 'x18', 'x1


100%|##########| 20/20 [00:15<00:00,  1.28it/s, fitness=648]

2024-08-23 09:06:48,497 - opytimizer.opytimizer — INFO — Optimization task ended.
2024-08-23 09:06:48,499 - opytimizer.opytimizer — INFO — It took 16.115137815475464 seconds.
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
2024-08-23 09:06:48,601 - opytimizer.spaces.search — INFO — Overriding class: Space -> SearchSpace.
2024-08-23 09:06:48,611 - opytimizer.core.space — DEBUG — Agents: 5 | Size: (331, 1) | Lower Bound: [-1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1

...

2024-08-23 09:37:22,443 - opytimizer.opytimizer — INFO — Optimization task ended.
2024-08-23 09:37:22,444 - opytimizer.opytimizer — INFO — It took 18.613837957382202 seconds.
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
2024-08-23 09:37:22,560 - opytimizer.spaces.search — INFO — Overriding class: Space -> SearchSpace.
2024-08-23 09:37:22,567 - opytimizer.core.space — DEBUG — Agents: 5 | Size: (161, 1) | Lower Bound: [-1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1



100%|##########| 20/20 [00:17<00:00,  1.15it/s, fitness=492]

2024-08-23 09:37:40,454 - opytimizer.opytimizer — INFO — Optimization task ended.
2024-08-23 09:37:40,459 - opytimizer.opytimizer — INFO — It took 17.860723972320557 seconds.
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 



100%|##########| 10/10 [30:00<00:00, 180.10s/it, fitness=236]

2024-08-23 09:37:40,567 - opytimizer.opytimizer — INFO — Optimization task ended.
2024-08-23 09:37:40,568 - opytimizer.opytimizer — INFO — It took 1884.5761544704437 seconds.


In [36]:
# Retrieve the best hyperparameter
best_hyperparams = opt_hyperparams.space.best_agent.position
best_hidden_units = int(best_hyperparams[0].item())

print(f"Best hidden units: {best_hidden_units}")

Best hidden units: 33


In [37]:
# Initialize the final model with the best hyperparameters found
final_model = TensorFlowNN(input_size=X_train.shape[1], hidden_size=best_hidden_units)

# Define the search space for weights and biases for the final optimization
n_weights_final = int(sum(np.prod(v.shape) for v in final_model.model.trainable_variables))
lower_bound_weights_final = -1.0 * np.ones(n_weights_final)
upper_bound_weights_final = 1.0 * np.ones(n_weights_final)

In [38]:
# Set up the SearchSpace for weight and bias optimization
space_weights_final = SearchSpace(
    n_agents=5,
    n_variables=n_weights_final,
    lower_bound=lower_bound_weights_final,
    upper_bound=upper_bound_weights_final
)

2024-08-23 09:37:40,616 - opytimizer.spaces.search — INFO — Overriding class: Space -> SearchSpace.
2024-08-23 09:37:40,627 - opytimizer.core.space — DEBUG — Agents: 5 | Size: (331, 1) | Lower Bound: [-1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -

In [40]:
# Define the objective function for the final weight and bias optimization
function_weights_final = Function(
    pointer=lambda weights: weight_objective_function(weights, final_model, X_train, y_train, X_val, y_val)
)

# Set simulated annealing hyperparameters
params = {
    't_initial': 0.1,    # Initial temperature
    't_final': 1e-4,     # Final temperature
    'alpha': 0.99        # Cooling rate
}

# Set up the simulated annealing optimizer for the final weights and biases
optimizer_weights_final = SA(params=params)
opt_weights_final = Opytimizer(space_weights_final, optimizer_weights_final, function_weights_final)

# Run the final optimization process for weights and biases
history_weights_final = opt_weights_final.start(n_iterations=10000)

2024-08-23 13:35:28,012 - opytimizer.core.function — INFO — Creating class: Function.
2024-08-23 13:35:28,015 - opytimizer.core.function — DEBUG — Function: <lambda> | Built: True.
2024-08-23 13:35:28,017 - opytimizer.core.function — INFO — Class created.
2024-08-23 13:35:28,019 - opytimizer.optimizers.science.sa — INFO — Overriding class: Optimizer -> SA.
2024-08-23 13:35:28,021 - opytimizer.core.optimizer — DEBUG — Algorithm: SA | Custom Parameters: {'t_initial': 0.1, 't_final': 0.0001, 'alpha': 0.99} | Built: True.
2024-08-23 13:35:28,022 - opytimizer.optimizers.science.sa — INFO — Class overrided.
2024-08-23 13:35:28,023 - opytimizer.opytimizer — INFO — Creating class: Opytimizer.
2024-08-23 13:35:28,025 - opytimizer.opytimizer — DEBUG — Space: <opytimizer.spaces.search.SearchSpace object at 0x7e40c68f4be0> | Optimizer: <opytimizer.optimizers.science.sa.SA object at 0x7e40c14475e0>| Function: <opytimizer.core.function.Function object at 0x7e40c1447010>.
2024-08-23 13:35:28,026 - op

100%|##########| 10000/10000 [2:38:41<00:00,  1.05it/s, fitness=10]

2024-08-23 16:14:10,431 - opytimizer.opytimizer — INFO — Optimization task ended.
2024-08-23 16:14:10,432 - opytimizer.opytimizer — INFO — It took 9522.401847839355 seconds.


In [41]:
# Retrieve the best weights and biases
best_weights_final = opt_weights_final.space.best_agent.position

# Apply the best weights to the final model
set_best_weights(best_weights_final, final_model)

In [42]:
# Evaluate the final model on the test set
y_pred_final = final_model.model.predict(X_test)
mse_final = np.mean((y_pred_final - y_test) ** 2)
print(f"MSE of the final optimized model on the test set: {mse_final:.4f}")

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
MSE of the final optimized model on the test set: 10.2284


# Early Stopping

In [17]:
# Early Stopping Callback
class EarlyStoppingCallback:
    def __init__(self, patience, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.best_fitness = None
        self.no_improvement_count = 0

    def on_iteration_end(self, current_fitness):
        if self.best_fitness is None:
            self.best_fitness = current_fitness
        elif current_fitness < self.best_fitness - self.min_delta:
            self.best_fitness = current_fitness
            self.no_improvement_count = 0
        else:
            self.no_improvement_count += 1

        return self.no_improvement_count >= self.patience

In [18]:
# Function to optimize weights and biases
def weight_objective_function(weights, model, X_train, y_train, X_val, y_val):
    # Set the weights and biases in the model
    new_weights = []
    start = 0
    for layer in model.model.layers:
        layer_weights = layer.get_weights()
        weight_shape = layer_weights[0].shape
        bias_shape = layer_weights[1].shape
        n_weight_params = np.prod(weight_shape)
        n_bias_params = np.prod(bias_shape)

        # Extract and reshape weights and biases
        weight_values = weights[start:start + n_weight_params].reshape(weight_shape)
        bias_values = weights[start + n_weight_params:start + n_weight_params + n_bias_params].reshape(bias_shape)

        new_weights.append(weight_values)
        new_weights.append(bias_values)

        start += n_weight_params + n_bias_params

    model.model.set_weights(new_weights)

    # Evaluate the model on the validation set
    y_val_pred = model.model.predict(X_val, verbose=0)
    val_loss = tf.keras.losses.MeanSquaredError()(y_val, y_val_pred).numpy()

    return float(val_loss)  # Ensure the return value is a float

In [19]:
# Function to set the best weights
def set_best_weights(weights, model):
    new_weights = []
    start = 0
    for layer in model.model.layers:
        layer_weights = layer.get_weights()
        weight_shape = layer_weights[0].shape
        bias_shape = layer_weights[1].shape
        n_weight_params = np.prod(weight_shape)
        n_bias_params = np.prod(bias_shape)

        # Extract and reshape weights and biases
        weight_values = weights[start:start + n_weight_params].reshape(weight_shape)
        bias_values = weights[start + n_weight_params:start + n_weight_params + n_bias_params].reshape(bias_shape)

        new_weights.append(weight_values)
        new_weights.append(bias_values)

        start += n_weight_params + n_bias_params

    # Set the weights and biases in the model
    model.model.set_weights(new_weights)

In [20]:
# Initialize the model
hidden_units = 10  # Example hidden units
model = TensorFlowNN(input_size=X_train.shape[1], hidden_size=hidden_units)

In [21]:
# Define the search space for weights and biases
n_weights = int(sum(np.prod(v.shape) for v in model.model.trainable_variables))
lower_bound_weights = -1.0 * np.ones(n_weights)
upper_bound_weights = 1.0 * np.ones(n_weights)

# Set up the SearchSpace for weight and bias optimization
space_weights = SearchSpace(
    n_agents=5,
    n_variables=n_weights,
    lower_bound=lower_bound_weights,
    upper_bound=upper_bound_weights
)

2024-08-24 05:29:30,212 - opytimizer.spaces.search — INFO — Overriding class: Space -> SearchSpace.
2024-08-24 05:29:30,218 - opytimizer.core.space — DEBUG — Agents: 5 | Size: (101, 1) | Lower Bound: [-1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.] | Upper Bound: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1.] | Mapping: ['x0', 'x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8',

In [22]:
# Define the objective function for weight optimization
function_weights = Function(
    pointer=lambda weights: weight_objective_function(weights, model, X_train, y_train, X_val, y_val)
)

# Set up the simulated annealing optimizer for weights and biases
params_inner = {
    't_initial': 0.1,
    't_final': 1e-4,
    'alpha': 0.99
}
optimizer_weights = SA(params=params_inner)
opt_weights = Opytimizer(space_weights, optimizer_weights, function_weights)

2024-08-24 05:29:46,605 - opytimizer.core.function — INFO — Creating class: Function.
2024-08-24 05:29:46,610 - opytimizer.core.function — DEBUG — Function: <lambda> | Built: True.
2024-08-24 05:29:46,611 - opytimizer.core.function — INFO — Class created.
2024-08-24 05:29:46,613 - opytimizer.optimizers.science.sa — INFO — Overriding class: Optimizer -> SA.
2024-08-24 05:29:46,615 - opytimizer.core.optimizer — DEBUG — Algorithm: SA | Custom Parameters: {'t_initial': 0.1, 't_final': 0.0001, 'alpha': 0.99} | Built: True.
2024-08-24 05:29:46,616 - opytimizer.optimizers.science.sa — INFO — Class overrided.
2024-08-24 05:29:46,618 - opytimizer.opytimizer — INFO — Creating class: Opytimizer.
2024-08-24 05:29:46,619 - opytimizer.opytimizer — DEBUG — Space: <opytimizer.spaces.search.SearchSpace object at 0x7af8bb5ffee0> | Optimizer: <opytimizer.optimizers.science.sa.SA object at 0x7af8bbf09c30>| Function: <opytimizer.core.function.Function object at 0x7af8bb261480>.
2024-08-24 05:29:46,621 - op

In [26]:
# Initialize Early Stopping Callback
callback = EarlyStoppingCallback(patience=100, min_delta=1e-8)

# Run the optimization process with early stopping
for iteration in range(10000):  # Maximum number of iterations
    opt_weights.start(n_iterations=1)
    current_fitness = opt_weights.space.best_agent.fit

    if callback.on_iteration_end(current_fitness):
        print(f"Early stopping at iteration {iteration+1} with best fitness {callback.best_fitness:.4f}")
        break

2024-08-24 05:31:12,694 - opytimizer.opytimizer — INFO — Starting optimization task.


100%|##########| 1/1 [00:00<00:00,  1.27it/s, fitness=508]

2024-08-24 05:31:13,861 - opytimizer.opytimizer — INFO — Optimization task ended.
2024-08-24 05:31:13,864 - opytimizer.opytimizer — INFO — It took 1.1655464172363281 seconds.
2024-08-24 05:31:13,865 - opytimizer.opytimizer — INFO — Starting optimization task.



100%|##########| 1/1 [00:00<00:00,  1.31it/s, fitness=508]

2024-08-24 05:31:15,017 - opytimizer.opytimizer — INFO — Optimization task ended.
2024-08-24 05:31:15,019 - opytimizer.opytimizer — INFO — It took 1.150109052658081 seconds.
2024-08-24 05:31:15,021 - opytimizer.opytimizer — INFO — Starting optimization task.


...

2024-08-24 05:36:54,474 - opytimizer.opytimizer — INFO — Optimization task ended.
2024-08-24 05:36:54,475 - opytimizer.opytimizer — INFO — It took 1.2854242324829102 seconds.
2024-08-24 05:36:54,478 - opytimizer.opytimizer — INFO — Starting optimization task.



100%|##########| 1/1 [00:00<00:00,  1.20it/s, fitness=458]

2024-08-24 05:36:55,727 - opytimizer.opytimizer — INFO — Optimization task ended.
2024-08-24 05:36:55,728 - opytimizer.opytimizer — INFO — It took 1.2462103366851807 seconds.
2024-08-24 05:36:55,729 - opytimizer.opytimizer — INFO — Starting optimization task.



100%|##########| 1/1 [00:00<00:00,  1.23it/s, fitness=458]

2024-08-24 05:36:56,941 - opytimizer.opytimizer — INFO — Optimization task ended.
2024-08-24 05:36:56,944 - opytimizer.opytimizer — INFO — It took 1.208970069885254 seconds.
Early stopping at iteration 282 with best fitness 458.3942


In [27]:
# Retrieve the best weights and biases found
best_weights = opt_weights.space.best_agent.position

# Apply the best weights to the model
set_best_weights(best_weights, model)

In [28]:
# Evaluate the final model on the test set
y_pred_final = model.model.predict(X_test)
mse_final = np.mean((y_pred_final - y_test) ** 2)
print(f"MSE of the final optimized model on the test set: {mse_final:.4f}")

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
MSE of the final optimized model on the test set: 382.9648
